# Finding the k in k-anonymity
All the other suppresion methods rely on a specific k. This notebook is all about finding the specific k we wanna use for each dataset

In [12]:
from anjana.anonymity import k_anonymity
from masking_effects_on_xai_techniques.hierarchy import generate_hierarchy
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [2]:
path = "../data/adult/"

In [3]:
data = pd.read_csv(path + "data.csv")
len_data = len(data)

# Clean the data
data['income'] = data['income'].str.rstrip('.')
data.dropna(inplace=True)
data.drop(columns=['education-num', 'fnlwgt'], inplace=True)

len_clean_data = len(data)
print(f"Dropped {len_data - len_clean_data} rows")
data.to_csv(path + "clean.csv", index=False)

Dropped 1221 rows


In [4]:
hierarchies = {
    "age": dict(pd.read_csv("../hierarchies/age.csv", header=None)),
    "education": dict(pd.read_csv("../hierarchies/education.csv", header=None)),
    "marital-status": dict(pd.read_csv("../hierarchies/marital.csv", header=None)),
    "occupation": dict(pd.read_csv("../hierarchies/occupation.csv", header=None)),
    "sex": dict(pd.read_csv("../hierarchies/sex.csv", header=None)),
    "native-country": dict(pd.read_csv("../hierarchies/country.csv", header=None)),
    "workclass": dict(pd.read_csv("../hierarchies/workclass.csv", header=None)),
    "relationship": dict(pd.read_csv("../hierarchies/relationship.csv", header=None)),
    "race": dict(pd.read_csv("../hierarchies/race.csv", header=None)),
    "capital-gain": generate_hierarchy(data['capital-gain'], 8),
    "capital-loss": generate_hierarchy(data['capital-loss'], 6),
    "hours-per-week": generate_hierarchy(data['hours-per-week'], 5),
}

In [5]:
quasi_ident = list(data.columns)
quasi_ident.remove('race')
quasi_ident.remove('income')

ident = ['race'] # Making race quasi identifier make k much larger
sens_att = 'income'

In [11]:
k_list = [2 ** n for n in range(1,9)]
supp_level = 20
annon_data = []
for k in k_list: 
    print(f"Anonymizing for k={k}")
    anon_df = k_anonymity_inner(
        data, ident, quasi_ident, k, supp_level, hierarchies
    )
    annon_data.append(anon_df)
print("Done anonymizing")

Anonymizing for k=2
Anonymizing for k=4
Anonymizing for k=8
Anonymizing for k=16
Anonymizing for k=32
Anonymizing for k=64
Anonymizing for k=128
Anonymizing for k=256
Done anonymizing


In [14]:
for k, (_,_,df) in zip(k_list, annon_data):
    df.to_csv(f"../data/adult/k_anonymity/{k}.csv", index=False)

AttributeError: 'dict' object has no attribute 'to_csv'

In [ ]:
predictors = []
data_split = []
for file_name, adult_prep in zip(file_names, data):
    print(file_name)
    X_train, X_test, y_train, y_test = train_test_split(
        adult_prep.X, adult_prep.y, test_size=0.4, random_state=0
    )
    clf = MLPClassifier(
        solver="sgd", alpha=1e-5, hidden_layer_sizes=(10), random_state=1
    )
    clf.fit(adult_prep.one_hot_encoder.transform(np.array(X_train)), y_train)
    print(
        clf.score(
            adult_prep.one_hot_encoder.transform(np.array(X_test)), np.array(y_test)
        )
    )
    predictors.append(clf)
    data_split.append((X_train, X_test, y_train, y_test))